In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib

In [7]:
#assuming uds qualified is assumed to have category in y_actual else use the below function
# def getCat(x):
#     if x > 0 and x <= 7:
#         return 'LOW'
#     elif x > 7 and x <= 21:
#         return 'MEDIUM'
#     elif x > 21 and x <= 60:
#         return 'HIGH'
#     else:
#         return 'VERY HIGH'
# finalData['y_actual'] = finalData['ONSET_DAYS'].apply(lambda x: getCat(x))

uds_qualified = pd.read_csv("/project/hackathon/hackers06/shared/finalDatafromAkshay/FinalData.csv")
uds_insurance = pd.read_excel("/project/hackathon/hackers06/shared/uds_insurance.xlsx")
patHsp = pd.read_excel('/project/hackathon/hackers06/shared/pat_hsp.xlsx')

## Process pat_hsp

In [8]:
patHsp['HOSP_DISCH_TIME'] = pd.to_datetime(patHsp['HOSP_DISCH_TIME'])
patHsp['EXP_DISCHARGE_DATE'] = pd.to_datetime(patHsp['EXP_DISCHARGE_DATE'])
patHsp['HOSP_ADMSN_TIME'] = pd.to_datetime(patHsp['HOSP_ADMSN_TIME'])
patHsp['HOSP_DISCH_DATE'] = [d.date() for d in patHsp['HOSP_DISCH_TIME']]
patHsp['EXP_DISCHARGE_DATE'] = [d.date() for d in patHsp['EXP_DISCHARGE_DATE']]
patHsp['HOSP_ADMSN_TIME'] = [d.date() for d in patHsp['HOSP_ADMSN_TIME']]
patHsp['LOS_HOSP'] = patHsp['HOSP_DISCH_DATE'] - patHsp['HOSP_ADMSN_TIME']
patHsp['LOS_HOSP'] = [d.days for d in patHsp['LOS_HOSP']]
patHsp['DISCH_DELEY'] = patHsp['HOSP_DISCH_DATE'] - patHsp['EXP_DISCHARGE_DATE']
patHsp['DISCH_DELEY'] = [d.days for d in patHsp['DISCH_DELEY']]

In [9]:
patHsp = patHsp[['SRC', 'PATIENT_NUM', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'EXP_DISCHARGE_DATE',
       'EXP_LEN_OF_STAY', 'ADMIT_SOURCE_C', 'ADMIT_SOURCE', 'ADM_EVENT_ID', 'ADT_PAT_CLASS_C', 
       'ADT_PAT_CLASS', 'HOSP_SERV_C', 'HOSP_SERVICE', 'DISCH_DISP_C', 'DISCH_DISP', 
       'ACCOMMODATION_C', 'HOSPITAL_AREA_ID', 'POS_NAME', 'HOSP_DISCH_DATE', 'LOS_HOSP','DISCH_DELEY']]


## Merge uds_qualified and insurance

In [13]:
uds_qualified['HEMISPHERIC_STROKE'] = uds_qualified['ADM_IMPAIR_CODE'].apply(lambda x: int(x in [1.1, 1.2]))
qualifiedAndInsurance = pd.merge(uds_qualified,
                 uds_insurance,
                 on="IHAR",
                 how="left")

In [14]:
def calculateInsuranceCategories(merged):
    merged['PRIM_PAYOR_SRC'] =  merged.apply(lambda x: x['PRIM_PAYOR_SRC'] if x['PRIM_PAYOR_SRC'] != 'NOT LISTED' else x['PRIM_PAYOR_SRC_DETAIL'], axis=1)
    merged['MEDICARE_FEE_FOR_SERVICE'] = 0
    merged['COMMERCIAL_AND_ADVANTAGE'] = 0
    merged['AGENCY'] = 0
    immediateList = ['MEDICARE FEE FOR SERVICE'] # ~1
    priorAuthList = ['MCO HMO', 'MEDICARE ADVANTAGE', 'COMM', 'BCBS'] # ~3
    # remaining = agency ~7
    merged['MEDICARE_FEE_FOR_SERVICE'] = merged.apply(lambda x: 1 if x['PRIM_PAYOR_SRC'] in immediateList else 0, axis=1)
    merged['COMMERCIAL_AND_ADVANTAGE'] = merged.apply(lambda x: 1 if x['PRIM_PAYOR_SRC'] in priorAuthList else 0, axis=1)
    merged['AGENCY'] = merged.apply(lambda x: 1 if x['PRIM_PAYOR_SRC'] not in priorAuthList and x['PRIM_PAYOR_SRC'] not in immediateList  else 0, axis=1)

In [15]:
calculateInsuranceCategories(qualifiedAndInsurance)


In [1]:
# modelData = merged[["IHAR", "MEDICARE_FEE_FOR_SERVICE", "COMMERCIAL_AND_ADVANTAGE", "AGENCY", "HEMISPHERIC_STROKE", "EXP_LOS", "y_actual"]]
# matplotlib.rcParams["figure.dpi"] = 300
# sns.scatterplot(x='ONSET_DAYS', y='ACTUAL_LOS', hue='y_actual', data=merged)
# merged['log_onset'] = np.log(merged['ONSET_DAYS'])
# sns.scatterplot(x='log_onset', y='ACTUAL_LOS', hue='y_actual', data=merged)

## Merge uds_insurance+uds_qualified with patHsp

In [23]:
df1 = uds_qualified[['IHAR','HOSP_ADMSN_TIME','PATIENT_NUM','y_actual']]
df2=patHsp[['HOSP_SERVICE','PATIENT_NUM','LOS_HOSP','DISCH_DELEY','HOSP_DISCH_TIME']]
modelData = qualifiedAndInsurance[["IHAR", "MEDICARE_FEE_FOR_SERVICE", "COMMERCIAL_AND_ADVANTAGE", "AGENCY", "HEMISPHERIC_STROKE", "EXP_LOS", "y_actual"]]
df_final=modelData.merge(df1,on='IHAR',how='left')

In [24]:
merged=df_final.merge(df2,on='PATIENT_NUM',how='left')
merged=merged.drop_duplicates()

In [26]:
merged=merged.reset_index(drop=True)
diff=[]
for enc in range(merged.shape[0]):
    thisadmtime=pd.to_datetime(merged['HOSP_ADMSN_TIME'][enc])
    thisdistime=pd.to_datetime(merged['HOSP_DISCH_TIME'][enc])
    try:
        thisdiff=((thisadmtime-thisdistime).total_seconds())/3600
        diff.append(thisdiff)
    except:
        thisdiff=-1
        diff.append(-1)
merged['diff']=diff
merged['diff']=merged['diff'].fillna(-1)

In [27]:
patdf=merged[(merged['diff']<12) & (merged['diff']>=0)]

In [28]:
patdf=patdf[['HOSP_SERVICE','LOS_HOSP','DISCH_DELEY','HOSP_DISCH_TIME','IHAR','diff']]

In [29]:
patdf=patdf.reset_index(drop=True)

In [30]:
final_df=df_final.merge(patdf,on='IHAR',how='left')

In [32]:
final_df['HOSP_SERVICE']=final_df['HOSP_SERVICE'].fillna('unknown')

In [33]:
services=[]
Neurosurgery=['Neurosurgery']
Neurology=['Neurology','Neurocritical Care']
Internal_Medicine=['Internal Medicine']
Orthopaedics=['Orthopaedics']
Pulmonary=['Pulmonary ']
Cardiology=['Cardiology']
Oncology=['Hematology-Oncology ']
Surgery=['Surgery','Surgery, Colon & Rectal','Oral Surgery','Plastic Surgery','CVTS',
         'Surgery, Surgical Transplantation','Surgery, Surgical Oncology',
        'Surgery, Vascular',
        ]
OtherDisease=['Bone Marrow Transplant','Urology','Otolaryngology','Geriatrics']

for service in range(final_df.shape[0]):
    thisservice=final_df['HOSP_SERVICE'][service]
    if thisservice in Neurosurgery:
        services.append('Neurosurgery')
    elif thisservice in Neurology:
        services.append('Neurology')
    elif thisservice in Internal_Medicine:
        services.append('Internal Medicine')
    elif thisservice in Orthopaedics:
        services.append('Orthopaedics')
    elif thisservice in Pulmonary:
        services.append('Pulmonary')
    elif thisservice in Cardiology:
        services.append('Cardiology') 
    elif thisservice in Oncology:
        services.append('Oncology')
    elif thisservice in Surgery:
        services.append('Surgery')
    elif thisservice in OtherDisease:
        services.append('Other Disease')
    else:
        services.append(thisservice)    

In [34]:
final_df['hosp_services']=services

In [35]:
final_df['LOS_HOSP']=final_df['LOS_HOSP'].fillna(np.nanmean(final_df['LOS_HOSP'].values))

In [36]:
final_df['DISCH_DELEY']=final_df['DISCH_DELEY'].fillna(np.nanmean(final_df['DISCH_DELEY'].values))

In [37]:
final_df=final_df.reset_index(drop=True)

In [38]:
final_df.drop('y_actual_x',axis=1,inplace=True)

In [39]:
final_df = final_df.rename(columns={'y_actual_y': 'y_actual'})

In [40]:
final_df=pd.get_dummies(final_df,columns=['hosp_services'],drop_first=True)

In [41]:
IRF_Data_Stratified = final_df

In [44]:
df_pat=pd.read_excel('/project/hackathon/hackers06/shared/pat_hsp.xlsx')
df_pat.drop('IHAR',axis=1,inplace=True)
merged=IRF_Data_Stratified.merge(df_pat,on='PATIENT_NUM',how='left')

In [48]:
merged=merged.drop_duplicates()

In [49]:
uds_qualified=uds_qualified[['IRF','CMG_RELATIVE_Weight','IHAR']]

In [50]:
final=merged.merge(uds_qualified,on='IHAR',how='left')

In [51]:
final.drop(['HOSP_ADMSN_TIME_x','HOSP_SERVICE_x','HOSP_DISCH_TIME_x','diff',
            'SRC','ADT_ARRIVAL_TIME', 'HOSP_ADMSN_TIME_y',
       'HOSP_DISCH_TIME_y', 'EXP_ADMISSION_TIME', 'EXP_DISCHARGE_DATE',
          'ADMIT_SOURCE_C', 'INP_ADM_DATE',
       'ED_EPISODE_ID', 'ED_DISP_TIME', 'ED_DISPOSITION_C', 'ED_DISP',
       'ADM_EVENT_ID', 'ADT_PAT_CLASS_C','TYPE_OF_BED_C','HOSP_SERV_C',
            'DISCH_DISP_C', 'DISCH_DEST_C','POS_NAME','HOSP_SERVICE_y',
            'EXP_LEN_OF_STAY' ,'BED_TYPE','DISCH_DEST','PATIENT_NUM','DEPARTMENT_ID',
            'ACCOMMODATION_C'
           ],axis=1,inplace=True)

In [52]:
final.fillna('unknown',inplace=True)

In [53]:
cat_col=['ADMIT_SOURCE', 'ADT_PAT_CLASS', 'DISCH_DISP', 'HOSPITAL_AREA_ID',
       'IRF']

In [54]:
final = pd.get_dummies(final,columns=cat_col,drop_first=True)

In [56]:
final #similar to IFAR we had

,IHAR,MEDICARE_FEE_FOR_SERVICE,COMMERCIAL_AND_ADVANTAGE,AGENCY,HEMISPHERIC_STROKE,EXP_LOS,y_actual,LOS_HOSP,DISCH_DELEY,hosp_services_Digestive and Liver Disease,...,HOSPITAL_AREA_ID_773.0,HOSPITAL_AREA_ID_833.0,HOSPITAL_AREA_ID_834.0,HOSPITAL_AREA_ID_923.0,HOSPITAL_AREA_ID_5050.0,HOSPITAL_AREA_ID_5053.0,HOSPITAL_AREA_ID_55000.0,HOSPITAL_AREA_ID_unknown,IRF_Qualifying Comorbid condition,IRF_Qualifying Etiologic diagnosis
0,301895713,0,1,0,1,24,HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
1,301908121,0,1,0,0,16,HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,1,0
2,301909531,1,0,0,0,22,VERY HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,301913503,0,1,0,0,10,MEDIUM,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
4,301914385,1,0,0,0,22,LOW,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
5,301914808,0,1,0,0,19,HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
6,302025004,1,0,0,0,11,LOW,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,1
7,302039650,1,0,0,0,18,HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
8,302044489,1,0,0,0,13,LOW,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,1,0
9,302049358,0,1,0,0,10,HIGH,9.951396,0.0,0,...,0,0,0,0,0,0,0,1,0,0
